In [1]:
import tweepy
import re
import emoji #!pip install emoji
from datetime import datetime
import mysql.connector
from mysql.connector import Error
from textblob import TextBlob


In [2]:
consumerKey = "erm0vjWKHjKM0ky*****"
consumerSecret = "EVB5y1R12xlIM3a2CHCgS7Obnq****"
accessToken = "1482385434500083715-86r8zXO4u5****"
accessTokenSecret = "KTX7tCrMDk2EvLOEQCOjo0TQ*********"
auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
auth.set_access_token(accessToken, accessTokenSecret)
api = tweepy.API(auth)

In [3]:
def tweetsparse(key, number):
    
    global keyword
    keyword= key
    noOfTweet = number
    tweets = tweepy.Cursor(api.search_tweets, q=keyword).items(noOfTweet) #connect with the Twitter API and proceed with the request
    
    global listOfTweets
    listOfTweets = [] #create a list to keep the data we prefer
    
    global EmptyTweets
    EmptyTweets = [] #create a list to keep if tweet containts empty only spaces values: #True or #False


    #parse the specific info for the tweets after checking if it is a retweet or not since the way Favorites are stored changes
    for tweet in tweets:    
    
        is_retweet = hasattr(tweet, 'retweeted_status') #check if tweet is a retweet
        if is_retweet == True :

            dict_ = {'User Name': tweet.user.screen_name, 
                 'Followers': tweet.user.followers_count,
                 'Tweets Count': tweet.user.statuses_count,
                 'Tweet Text': tweet.text,
                 'Tweet Created at': tweet.created_at,
                 'Location': tweet.user.location,
                 'Hashtags': tweet.entities['hashtags'],
                 'Retweets':tweet.retweet_count,
                 'Likes': tweet.retweeted_status.favorite_count}

        else:

            dict_ = {'User Name': tweet.user.screen_name, 
                 'Followers': tweet.user.followers_count,
                 'Tweets Count': tweet.user.statuses_count,
                 'Tweet Text': tweet.text,
                 'Tweet Created at': tweet.created_at,
                 'Location': tweet.user.location,
                 'Hashtags': tweet.entities['hashtags'],
                 'Retweets':tweet.retweet_count,
                 'Likes': tweet.favorite_count } 


        listOfTweets.append(dict_)
    
    #clean the data we extracted
    
    for i in range(0,len(listOfTweets)):
        
        #insert @ in the username using for loop in our list
        listOfTweets[i]['User Name'] = re.sub(r'([\w.-]+)', r'@\1', listOfTweets[i]['User Name'])
        
        #Clear RT and user who made the initial tweet if it was a retweet
        matchRT = re.search(r'RT', listOfTweets[i]['Tweet Text'])
        if matchRT:
            listOfTweets[i]['Tweet Text'] = re.sub(r'RT', "", listOfTweets[i]['Tweet Text'])
            listOfTweets[i]['Tweet Text'] = re.sub(r'@([\w.-]+:)', "", listOfTweets[i]['Tweet Text'])

        #Clear all a) mentions b) hashtags to text c) URLs and d) \n\n from a tweet

        matchMentions = re.findall(r'@([\w.-]+)', listOfTweets[i]['Tweet Text'])
        if matchMentions:
            listOfTweets[i]['Tweet Text'] = re.sub(r'@([\w.-]+)', "", listOfTweets[i]['Tweet Text'])

        #Match a Hashtag and remove the #
        matchHashtags = re.findall(r'#([\w.-]+)', listOfTweets[i]['Tweet Text'])
        if matchHashtags:
            listOfTweets[i]['Tweet Text'] = re.sub(r'#([\w.-]+)', r'\1', listOfTweets[i]['Tweet Text'])
     
        #Regex for matching a URL
        regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"

        #Match a URL
        matchURL = re.findall(regex, listOfTweets[i]['Tweet Text'])
        if matchURL:
            listOfTweets[i]['Tweet Text'] = re.sub(regex, "", listOfTweets[i]['Tweet Text'])
        
        #Match a \n\n
        matchURL = re.findall('([^.])\.?\n\n', listOfTweets[i]['Tweet Text'])
        if matchURL:
            listOfTweets[i]['Tweet Text'] = re.sub('([^.])\.?\n\n', "", listOfTweets[i]['Tweet Text'])
        
               
        #Clear all emojis from text and location after inserting a regex
        
        emoj = re.compile("(["
        "\U0001F1E0-\U0001F1FF"  # flags (iOS)
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F700-\U0001F77F"  # alchemical symbols
        "\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
        "\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
        "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        "\U0001FA00-\U0001FA6F"  # Chess Symbols
        "\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        "\U00002702-\U000027B0"  # Dingbats
        "])")
        
        matchEmojis = re.findall(emoj, listOfTweets[i]['Tweet Text'])
        if matchEmojis:
            listOfTweets[i]['Tweet Text'] = re.sub(emoj, "", listOfTweets[i]['Tweet Text'])
        
        LmatchEmojis = re.findall(emoj, listOfTweets[i]['Location'])
        if LmatchEmojis:
            listOfTweets[i]['Location'] = re.sub(emoj, "", listOfTweets[i]['Location'])
            
            
        #add hashtags on the 'hashtags'for each tweet and convert it to a format for our database
        for j in range(0,len(listOfTweets[i]['Hashtags'])):
            listOfTweets[i]['Hashtags'][j]['text'] = re.sub(r'([\w.-]+)', r'#\1', listOfTweets[i]['Hashtags'][j]['text'])
        lHash = []
        for j in range(0,len(listOfTweets[i]['Hashtags'])):
            lHash.append(listOfTweets[i]['Hashtags'][j]['text'])
        listOfTweets[i]['Hashtags'] = " ".join(lHash)
        lHash
        
        #Convert the dates
        listOfTweets[i]['Tweet Created at'] = datetime.strftime(listOfTweets[i]['Tweet Created at'], '%Y-%m-%d')
        
        #Check now if the text is empty - contains only spaces
        EmptyTweets.append(listOfTweets[i]['Tweet Text'].isspace())
        
    #Now Delete the tweets that containt only spaces inside the text
    for i in range(len(EmptyTweets)-1, 0, -1):
        if EmptyTweets[i] == True:
            del listOfTweets [i]
    
    #Finally Remove the duplicate posts of the same user. Meaning the user has made the same post multiple times
    seen = set()
    
    global flistOfTweets
    flistOfTweets = []

    for dic in listOfTweets:
        key = (dic['User Name'], dic['Tweet Text'])
        if key in seen:
            continue

        flistOfTweets.append(dic)
        seen.add(key)
    
    print('After removing Empty:', len(EmptyTweets))
    print('After removing Duplicates:', len(flistOfTweets))

In [37]:
def databaseupload(l, keyword):
    
    global tablename
    
    listOfTweets = l
    
    db = mysql.connector.connect(host='localhost', database='tweets', user='root', password= '****', auth_plugin='mysql_native_password') 
    if db.is_connected():
        print("CONNECTED TO MYSQL DATABASE!") 
        cur = db.cursor()
        
    #Get the table name based on the keyword. First check if there is a hashtag
    checkhashtag = re.findall(r'#', keyword)
    if checkhashtag:
        tablename = re.sub(r'#', '', keyword)
    else:
        tablename = keyword
    
    #create the table dynamically based on the keyword the user inserted and after checking if the table already exists
    q = """SHOW TABLES"""
    cur.execute(q)
    results = cur.fetchall()
    results_list = [item[0] for item in results] # Conversion to list of str
    
    #Check if the table doesn't exist in our database create one
    if tablename not in results_list:
        q = "CREATE TABLE {} (id int AUTO_INCREMENT, user_name VARCHAR(50), followers INT, tweets_count INT, tweet_text VARCHAR(300), tweet_created_at DATE, location VARCHAR(200), hashtags VARCHAR(300), retweets INT, likes INT, PRIMARY KEY (id));".format(tablename)
        cur.execute(q)
        
        #we have tried to remove all the emojis but in the case that some tweets still have emojis we change the database so as to be able to accept them
        q = "ALTER TABLE {} MODIFY tweet_text BLOB;".format(tablename)
        cur.execute(q)
        
        q = "ALTER TABLE {} MODIFY location BLOB;".format(tablename)
        cur.execute(q)
        
    #insert data into database 
    for i in range(0,len(listOfTweets)):
        user_name = str(listOfTweets[i]['User Name'])
        followers =  listOfTweets[i]['Followers']
        tweets_Count = listOfTweets[i]['Tweets Count']
        tweet_text = str(listOfTweets[i]['Tweet Text'])
        tweet_created_at = listOfTweets[i]['Tweet Created at']
        location = str(listOfTweets[i]['Location'])
        hashtags = str(listOfTweets[i]['Hashtags'])
        retweets = listOfTweets[i]['Retweets']
        likes = listOfTweets[i]['Likes']

        q= "INSERT INTO {} (user_name, followers, tweets_count, tweet_text, tweet_created_at, location, hashtags, retweets, likes) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)".format(tablename)
        cur.execute(q, (user_name, followers, tweets_Count, tweet_text, tweet_created_at, location, hashtags, retweets, likes))

    db.commit()
    db.close()

In [49]:
def SentAnalysis (table):
    
    tablename = table
    
    #connect to the database
    db = mysql.connector.connect(host='localhost', database='tweets', user='root', password= '****', auth_plugin='mysql_native_password') 
    if db.is_connected():
        print("CONNECTED TO MYSQL DATABASE!") 
        cur = db.cursor()
        
    #check if the column "sentiment" exists in the table else add the column
    
    q = "SHOW COLUMNS from {} LIKE 'sentiment';".format(tablename)
    cur.execute(q)
    results = cur.fetchall()
    results_list = [item[0] for item in results] # Conversion to list of str

    if 'sentiment' not in results_list:
        q = "ALTER TABLE {} ADD sentiment varchar(20);".format(tablename)
        cur.execute(q)
        
    #Get the texts from Database for  analysis
    q= "SELECT tweet_text FROM {}".format(tablename)
    cur.execute(q)
    
    #append them on a list after converting them to strings
    tweet_list = []

    for tweet in cur:
        tweet_list.append(str(tweet))          
    
    for i in range(len(tweet_list)):
    
        analysis = TextBlob(tweet_list[i]) #use of TextBlob
        polarity = analysis.sentiment.polarity #polarity

        #results based on the polarity
        
        if polarity < - 0.05:
            text = 'negative'

        elif polarity > 0.05:
            text = 'positive'

        else:
            text = 'neutral'

        id = i+1
        
        #append the result of each tweet analysis on the table
        q = "UPDATE {} SET sentiment= %s WHERE id = %s ".format(tablename)
        cur.execute(q, (text, id))        
        db.commit()

    db.close()

## Get the tweets and upload them into Database

### Bucks

In [38]:
tweetsparse('#Bucks', 1000)

After removing Empty: 1000
After removing Duplicates: 999


In [39]:
databaseupload(flistOfTweets, keyword)

CONNECTED TO MYSQL DATABASE!


### Lakers

In [40]:
tweetsparse('#Lakers', 1000)

After removing Empty: 1000
After removing Duplicates: 972


In [41]:
databaseupload(flistOfTweets, keyword)

CONNECTED TO MYSQL DATABASE!


### Celtics

In [32]:
tweetsparse('#Celtics', 1000)

After removing Empty: 1000
After removing Duplicates: 995


In [33]:
databaseupload(flistOfTweets, keyword)

CONNECTED TO MYSQL DATABASE!


## Perform Sentiment Analysis for the Table


In [50]:
SentAnalysis ('Celtics')

CONNECTED TO MYSQL DATABASE!


In [51]:
SentAnalysis ('Lakers')

CONNECTED TO MYSQL DATABASE!


In [52]:
SentAnalysis ('Bucks')

CONNECTED TO MYSQL DATABASE!
